# Creation of eval dataset of prose

Creates `prosa_test_text.csv` file with prose test dataset.

In [ ]:
%pip install numpy pandas nltk

In [24]:
import numpy as np 
import pandas as pd 
import nltk
from nltk.tokenize import sent_tokenize
import os

nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Марина\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Марина\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

To use this notebook download source dateset of Russian Literature from https://www.kaggle.com/datasets/d0rj3228/russian-literature as zip file. Then unzip it to './archive/'.

Functions for reading files and extracting chunks.

In [25]:
def read_first(filename, count_lines):
    lines = []
    with open(filename, 'r', encoding='utf-8') as file:
            for i, line in enumerate(file):
                if i >= count_lines:
                    break
                lines.append(line.rstrip())  # Удаляем лишние символы новой строки
    return lines

def split_text_into_chunks(paragraphs, min_length=210, max_length=310):
    chunks = []
    for paragraph in paragraphs:
        if not paragraph.strip():
            continue
        
        sentences = sent_tokenize(paragraph.strip())

        for sentence in sentences:
            sentence = sentence.strip()
            if len(sentence) <= max_length and len(sentence) >= min_length:
                chunks.append(sentence)

    return chunks

Extracting chunks

In [26]:
chunks = {}
chunks_long = {}
for dirname, _, filenames in os.walk('./archive/prose/'):
    chunks[dirname] = []
    chunks_long[dirname] = []
    for filename in filenames:
        if filename.endswith('.txt'):
            paragraphs = read_first(os.path.join(dirname, filename), count_lines=20000)
            chunks[dirname].extend(split_text_into_chunks(paragraphs, min_length=100, max_length=140))

In [27]:
all_chunks = []
for k, v in chunks.items():
    all_chunks.extend(v)

In [28]:
len(all_chunks)

47396

Examples of chunks and their lens.

In [ ]:
for chunk in all_chunks[:10]:
    print(chunk, len(chunk))

Туристы влезают туда, крутят вверху носами, любуясь на виды, и лопочут дикие и ненужные речи об окрестных пейзажах на разноплеменных языках. 140
Колесо приводит в движение поршень, тянущийся по полям и холмам, под железнодорожной насыпью на несколько километров. 117
Жарко невыносимо, нивы наливаются, черешни краснеют по краям шоссе, никем не охраняемые, пирамидальные тополя в пыли. 117
Ну, конечно, объясняются в любви и целуются на скамейках, пользуясь сном "для здоровья" благоразумных европейцев. 113
И свидания над туманным озером еще больше нагоняют скуку, знаешь, что через месяц-два поплетешься по Невскому под мокрым снегом. 128
Это - высокие стены, туго набитые хворостом; под ними - деревянные резервуары, куда, серебристо звякая, без устали падает жидкая соль. 134
Они вдыхают соль ртами и носами и представляют комментарий к тексту: "если соль потеряет свою силу, кто сделает ее соленою". 124
По крайней мере, судя по их позам, лицам, пиджакам и юбкам, думается, что никакая сила в мир

Saving as a csv file and adding meter and rhyme scheme.

In [29]:
max_cap = 1000
texts = np.random.choice(all_chunks, size=max_cap, replace=False)

In [ ]:
df = pd.DataFrame({'text': texts,
                   'rhyme_scheme': ['AABB'] * 333 + ['ABAB'] * 333 + ['ABBA'] * 334,})
df.loc[::2, 'meter'] = 'choreios'
df.loc[1::2, 'meter'] = 'iambos'
df.to_csv('prosa_test_text.csv', index=False)
df

,text,rhyme_scheme,meter
0,Та же история повторилась с одной саксонской с...,AABB,choreios
1,"-- Ответьте сначала, -- сказал мне он, -- родс...",AABB,iambos
2,"Хорошенькая просыпается, обводит глазами публи...",AABB,choreios
3,"– Ах, милый князь, – воскликнул вдруг Евгений ...",AABB,iambos
4,"Перед тем, как идти в гости, она завязала Лаев...",AABB,choreios
...,...,...,...
995,"– Да, да… Я непременно хочу знать, что вы дела...",ABBA,iambos
996,"""За тобой пришел, красная девица; уводи ж меня...",ABBA,choreios
997,"Потом, когда всунете войлок, вложите пулю, – с...",ABBA,iambos
998,"Впрочем, в темноте трудно угадать направление ...",ABBA,choreios
